## Import các thư viện cần thiết

In [16]:
import os
import glob
import random
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

/Users/conquerormikrokosmos/miniforge3/envs/sudo-code-2025/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Kiểm tra đường dẫn đến dataset

In [2]:
data_path = "../data/raw/viwik18/dataset"

In [3]:
!ls ../data/raw/viwik18/dataset

viwik18_aa viwik18_ac viwik18_ae viwik18_ag viwik18_ai
viwik18_ab viwik18_ad viwik18_af viwik18_ah viwik18_aj


In [4]:
filename_list = os.listdir(data_path)
filename_list.sort()
filename_list

['viwik18_aa',
 'viwik18_ab',
 'viwik18_ac',
 'viwik18_ad',
 'viwik18_ae',
 'viwik18_af',
 'viwik18_ag',
 'viwik18_ah',
 'viwik18_ai',
 'viwik18_aj']

In [5]:
# Concat the filename with the data file path to get the full path of each file
filepath_list = [os.path.join(data_path, filename) for filename in filename_list]
filepath_list

['../data/raw/viwik18/dataset/viwik18_aa',
 '../data/raw/viwik18/dataset/viwik18_ab',
 '../data/raw/viwik18/dataset/viwik18_ac',
 '../data/raw/viwik18/dataset/viwik18_ad',
 '../data/raw/viwik18/dataset/viwik18_ae',
 '../data/raw/viwik18/dataset/viwik18_af',
 '../data/raw/viwik18/dataset/viwik18_ag',
 '../data/raw/viwik18/dataset/viwik18_ah',
 '../data/raw/viwik18/dataset/viwik18_ai',
 '../data/raw/viwik18/dataset/viwik18_aj']

In [6]:
# Read the first file with encoding UTF-8.
# Replace error bytes with a placeholder character.
with open(filepath_list[0], "r", encoding="utf-8", errors="replace") as f:
    text = f.read()
text[-1000:]

'hân vệ đại tướng quân và được ban quy phù đến đời vua trần anh tông ông được thăng tới chức điện súy thượng tướng quân tước quan nội hầu một người con gái của phạm ngũ lão hiệu là tĩnh huệ là thứ phi của vua anh tông   phạm ngũ lão đã ba lần cất quân đi trừng phạt sự xâm chiếm quấy nhiễu của quân ai lao vào các năm một hai chín bốn một hai chín bảy và một ba không một hai lần nam chinh đánh thắng quân chiêm thành vào năm một ba một hai buộc vua chiêm là chế chí phải xin hàng và một ba một tám vua chế năng phải bỏ chạy sang java   văn học  không chỉ có tài về quân sự mà ông còn để lại nhiều bài thơ về chí trai lòng yêu nước tiếc là hiện nay tác phẩm của ông chỉ còn lại hai bài là thuật hoài và vãn thượng tướng quốc công hưng đạo đại vương   đời tư  ông được trần hưng đạo gả con gái là quận chúa anh nguyên   qua đời  ngày một tháng một một năm một ba hai không phạm ngũ lão mất hưởng thọ sáu sáu tuổi vua trần minh tông nghỉ chầu năm ngày đây là một đặc ân của nhà vua đối với ông   nhân d

In [7]:
with open(filepath_list[1], "r", encoding="utf-8", errors="replace") as f:
    text = f.read()
text[:500]

'�n xã phù ủng dựng đền thờ ngay trên nền nhà cũ của ông ông cũng được phối thờ tại đền kiếp bạc chí linh hải dương tại đền thờ trần hưng đạo   tác phẩm  mặc dù chuyên lo việc quân song phạm ngũ lão vẫn thích đọc sách ngâm thơ thơ văn ông hiện chỉ còn lại hai bài thơ chữ hán   thuật hoài làm theo thể thất ngôn tứ tuyệt   vãn hưng đạo đại vương làm theo thể thất ngôn bát cú tuy nhiên có ý kiến cho rằng bài này là của bùi tông hoan   nhận định  nhà sử học lỗi lạc của nước nhà ở thế kỉ thứ xv là tiế'

### Vấn đề
Khi tải dữ liệu Wikipedia dạng _shard_ (chia thành nhiều file nhỏ), một ký tự UTF-8 có thể bị “cắt ngang” ở cuối file đầu tiên và tiếp tục ở đầu file tiếp theo. UTF-8 dùng từ 1–4 byte cho mỗi ký tự, nên nếu chỉ đọc từng file riêng lẻ, Python sẽ gặp `UnicodeDecodeError` vì nó thấy một byte mở đầu mà thiếu byte tiếp theo.

Cụ thể với `viwik18_aa`: byte cuối không đủ thông tin để tạo thành ký tự hợp lệ, dẫn đến lỗi decode.

### Giải pháp
Thay vì decode từng file một, ta nên nối tất cả các file ở mức byte (rb mode) rồi mới decode một lần. Như vậy các byte ký tự bị chia nhỏ ở ranh giới file sẽ được ghép lại đầy đủ, không còn gây lỗi.

In [8]:
# Create a pattern to the data files
file_pattern = os.path.join(data_path, "viwik18_*")

# Search for all data files in the data directory
files = sorted(glob.glob(file_pattern))

In [9]:
files

['../data/raw/viwik18/dataset/viwik18_aa',
 '../data/raw/viwik18/dataset/viwik18_ab',
 '../data/raw/viwik18/dataset/viwik18_ac',
 '../data/raw/viwik18/dataset/viwik18_ad',
 '../data/raw/viwik18/dataset/viwik18_ae',
 '../data/raw/viwik18/dataset/viwik18_af',
 '../data/raw/viwik18/dataset/viwik18_ag',
 '../data/raw/viwik18/dataset/viwik18_ah',
 '../data/raw/viwik18/dataset/viwik18_ai',
 '../data/raw/viwik18/dataset/viwik18_aj']

In [10]:
all_bytes = b"".join(open(f, "rb").read() for f in files)
text = all_bytes.decode("utf-8")
split_text = text.split("  ")

In [11]:
split_text[:100]

['trang chính',
 'internet society',
 'internet society hay isoc là một tổ chức quốc tế hoạt động phi lợi nhuận phi chính phủ và bao gồm các thành viên có trình độ chuyên ngành tổ chức này chú trọng đến tiêu chuẩn giáo dục và các vấn đề về chính sách với trên một bốn năm tổ chức thành viên và sáu năm không không không thành viên cá nhân isoc bao gồm những con người cụ thể trong cộng đồng internet mọi chi tiết có thể tìm thấy tại website của isoc',
 ' internet society nằm ở gần thủ đô washington dc hoa kỳ và geneva thụy sĩ số hội viên của nó bao gồm hơn một bốn năm tổ chức thành viên và hơn sáu năm không không không cá nhân thành viên còn có thể tự lập một chi nhánh của tổ chức tùy theo vị trí hoặc sở thích hiện nay tổ chức có tới chín không chi nhánh trên toàn thế giới',
 ' nhiệm vụ và mục đích hoạt động',
 'bảo đảm cổ vũ cho sự phát triển mở rộng và sử dụng internet được thuận lợi nhất cho mọi người trên toàn thế giới',
 ' xem thêm',
 ' lịch sử internet',
 'liên kết ngoài',
 ' isoc vi

In [12]:
# For each split, perform a .strip() on them
cleaned_splits = [s.strip() for s in split_text if s.strip()]

In [15]:
# Set random seed
random.seed(42)

# Randomize 100 splits
randomized_splits = random.sample(cleaned_splits, 100)
randomized_splits[:10]

['o herre một chín một tám recording by eleonora olson',
 'yeonnam dong',
 'để xoa dịu phe cánh nguyễn kính chiêu tông sai giết những người gièm pha trần chân trước đây nhưng kính vẫn tụ tập cướp phá như trước mạc đăng dung khuyên chiêu tông về bảo châu nhưng nội thần đỗ nhạc và nguyễn dư can ngăn đăng dung đem giết cả hai đưa chiêu tông cùng em là lê xuân về bảo châu sau khi chiêu tông rời đông kinh vĩnh hưng bá trịnh tuy cùng bọn giặc sơn tây lập lê bảng con của tĩnh tu công lê lộc lên ngôi đổi thành đại đức đế được nửa năm lại phế bảng mà lập lê do đổi thành thiên hiến đế cả hai đều là cháu bốn đời của cung vương lê khắc xương con thứ hai của thái tông văn hoàng',
 'hộ chiếu ngoại giao và công vụ',
 'catocala caesia',
 'symplocos trichoclada',
 'hoặc',
 'liên kết ngoài',
 'ba the hoaglands solution for hydroponic cultivation science in hydroponics truy cập ngày một tháng một không năm hai không một bốn',
 'acacia bombycina']

In [17]:
tokenizer = AutoTokenizer.from_pretrained("NlpHUST/vi-word-segmentation")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/vi-word-segmentation")

In [18]:
nlp = pipeline("token-classification", model=model, tokenizer=tokenizer)

Device set to use mps:0


In [21]:
example = "để xoa dịu phe cánh nguyễn kính chiêu tông sai giết những người gièm pha trần chân trước đây nhưng kính vẫn tụ tập cướp phá như trước mạc đăng dung khuyên chiêu tông về bảo châu nhưng nội thần đỗ nhạc và nguyễn dư can ngăn đăng dung đem giết cả hai đưa chiêu tông cùng em là lê xuân về bảo châu sau khi chiêu tông rời đông kinh vĩnh hưng bá trịnh tuy cùng bọn giặc sơn tây lập lê bảng con của tĩnh tu công lê lộc lên ngôi đổi thành đại đức đế được nửa năm lại phế bảng mà lập lê do đổi thành thiên hiến đế cả hai đều là cháu bốn đời của cung vương lê khắc xương con thứ hai của thái tông văn hoàng"
ner_results = nlp(example)
ner_results

example_tok = ""
for e in ner_results:
    if "##" in e["word"]:
        example_tok = example_tok + e["word"].replace("##","")
    elif e["entity"] =="I":
        example_tok = example_tok + "_" + e["word"]
    else:
        example_tok = example_tok + " " + e["word"]
print(example_tok)

 để xoa_dịu phe_cánh nguyễn kính chiêu_tông sai giết những người gièm_pha trần chân trước_đây nhưng kính vẫn tụ_tập cướp_phá như trước mạc đăng_dung khuyên chiêu_tông về bảo_châu nhưng nội_thần đỗ nhạc và nguyễn_dư can_ngăn đăng_dung đem giết cả hai đưa chiêu tông cùng em là lê_xuân về bảo_châu sau khi chiêu tông rời đông_kinh vĩnh_hưng bá trịnh_tuy cùng bọn giặc sơn_tây lập lê bảng con của tĩnh tu công lê lộc lên_ngôi đổi thành đại đức đế được nửa năm lại phế bảng mà lập lê do đổi thành thiên hiến đế cả hai đều là cháu bốn đời của cung vương lê khắc xương con thứ hai của thái_tông văn hoàng
